In [1]:
import pandas as pd
pd.set_option("display.precision", 10)
pd.set_option('display.max_rows', 40)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 300)
#pd.set_option('display.max_colwidth', -1)
from scipy import stats as st
import scipy.stats
import utils as ut
import numpy as np
from tqdm import tqdm
import math

### Data Loading & Settings

In [2]:
event_frame_size = 21 # Counts from the right in excel sheet 21 columns and sets the split between estimation and event window 
event_sizes = [3,7,11,21] # event window size. This number does not effect estimation window size only event window size
OMXPI_csv = "OMXPI.xlsx_Sheet1"
stock_csv = "Financial_Stock_Data"


In [4]:
for event_size in event_sizes:
    
    ### LOADING DATA ---------------------------------------------------  
    # Load OMXPI data
    omxp_index = pd.read_csv("data/"+OMXPI_csv+".csv")
    
    # load stock data
    stock_csv = "Financial_Stock_Data"
    df_dnb = pd.read_csv ("data/"+stock_csv+".csv", delimiter=",", header=None)

    
    ### STOCK Calculation ---------------------------------------------- 
    
    # Locate window split in table data 
    thres = df_dnb.shape[1]-event_frame_size

    # Get event window index values 
    lower_idx, upper_idx = ut.event_win_indexes(thres, event_frame_size, event_size)

    # Define variables
    stocks_data = pd.DataFrame()
    stocks_metadata = pd.DataFrame()
    
    # Each data entry takes up 3 rows so loop iterations are divide by 3
    for it in tqdm(range(int(df_dnb.shape[0]/3))):
        # Select stock(relevant rows) based on iterator(it) from table 
        df_stock = df_dnb.iloc[0+it*3:2+it*3]
        # convert to list
        stock = df_stock.values.tolist()

        # Check for nan values. If true then skip stock
        if ut.has_nan_values(stock[1], it): continue

        # Convert stock values from strings to floats
        stock[1][1:] = [float(i) for i in stock[1][1:]]

        # Get the respective OMXPI values
        OMXPI_values = ut.get_omxpi_values(omxp_index, stock[0])
        stock.append(OMXPI_values)

        # Get the stock return values
        stock_return_values = ut.stock_return_values(stock[1])
        stock.append(stock_return_values)

        # Get the market return values
        market_return_values = ut.market_return_values(stock[2])
        stock.append(market_return_values)

        # Convert newly created data back to dataframe
        df_stock = pd.DataFrame(stock)

        # Define window sizes 
        estimatation_window = ut.get_windows(df_stock, thres, event_size)
        est_win_size = estimatation_window.shape[1]-1

        # Get slope and intercept
        stock_return_estw = estimatation_window.iloc[[3]].values[0][2:].tolist()
        market_return_estw = estimatation_window.iloc[[4]].values[0][2:].tolist()
        intercept, slope, stderr = ut.linear_regression(market_return_estw, stock_return_estw)

        # Compute Standard deviation
        lst1 = df_stock.iloc[3][thres:].to_list()
        lst2 = df_stock.iloc[4][thres:].to_list()
        std_dev = pd.DataFrame(list(zip(lst1, lst2))).std()[0]

        # Compute Normal return
        market_return = df_stock.iloc[4].values.tolist()[1:]
        normal_return = ut.normal_return_values(market_return, slope, intercept)
        stock.append(normal_return)

        # Compute Abnormal return
        abnormal_return = np.subtract(stock[3][1:],stock[5][1:]).tolist()
        abnormal_return.insert(0,"AbnormalReturn")
        stock.append(abnormal_return)

        # Compute t-statistic R
        abnormal_return = stock[6][1:]
        t_stat_r = ut.t_stat_r(abnormal_return, stderr)
        stock.append(t_stat_r)

        # Put the stock data into a table again
        df_stock = pd.DataFrame(stock)

        # Get car value
        abnormalReturn_event_window = df_stock.iloc[6][lower_idx:upper_idx]
        car_value = abnormalReturn_event_window.sum()


        # Get t-stat CAR
        t_stat_car = car_value / (std_dev*(event_size)**(1/2))

        # Collect all stocks together in one table 
        # --------------------------------------------
        # Add stock meta-data to data table 
        row1 = ["df_stockRow", "df_stock#","intercept", "slope", "standard_error", "t_stat_car", "car_value", "eve_win_size"]
        row2 = [[(it*3), (it), intercept, slope, stderr, t_stat_car, car_value, event_size]]
        variables = pd.DataFrame(row2)
        variables.columns = row1

        # Collect all stock data 
        stocks_data = pd.concat([stocks_data, df_stock], axis=0)
        stocks_metadata = pd.concat([stocks_metadata, variables], axis=0)
        

    ### Pirate Calculations 1 ------------------------------------------ 
    df_AbnormalReturn = stocks_data.loc[stocks_data[0] == "AbnormalReturn"]

    # Means for each column
    aar_means = ut.mean(df_AbnormalReturn, lower_idx, upper_idx)

    # Standard deviation for each column
    aar_stds = ut.st_dev(df_AbnormalReturn, lower_idx, upper_idx)

    # 1-sample T-test and P-value
    aar_tstats, aar_pvalues = ut.t_test(df_AbnormalReturn, lower_idx, upper_idx)
    
    ### Pirate Calculations 2 ------------------------------------------ 
    
    oddvalues = {}
    # AAR Values 
    row1 = ["AAR", "standardDev", "Tstat", "Pvalue"]
    row2 = [aar_means, aar_stds, aar_tstats, aar_pvalues]
    variables = pd.DataFrame(row2)
    variables = variables.T
    variables.columns = row1

    # CAAR Values
    ev21 = variables["AAR"].sum()
    ev11 = variables["AAR"][5:-5].sum()
    ev5 = variables["AAR"][7:-7].sum()
    ev3 = variables["AAR"][9:-9].sum()
    CAAR = {"ev21": ev21, "ev11": ev11, "ev5": ev5, "ev3": ev3}
    oddvalues["caar_value"] = CAAR["ev21"]

    # CAAR standard deviations
    caar_std = {}
    carlist = []
    for key, caar in CAAR.items():
        for car in stocks_metadata["car_value"]:
            carlist.append((car-caar)**2)
        caar_std[key] = math.sqrt(sum(carlist)/len(carlist))
    oddvalues["caar_std"] = caar_std["ev21"]  

    for i in zip(list(CAAR.values()), list(caar_std.values())):  
        ttest = i[0]/i[1]
        oddvalues["caar_pvalue"] = scipy.stats.t.sf(ttest, df=df_dnb.shape[0]/3-1)
        break

    # WilCoxon test
    oddvalues["willCox_stat"], oddvalues["willCox_pvalue"] = scipy.stats.wilcoxon(stocks_metadata["car_value"])
    oddvalues = pd.DataFrame(oddvalues, index=[0])
    
    ### SAVE to EXCEL --------------------------------------------------  
    print("Saving to excel...")
    # Create a Pandas Excel writer
    path = "export/"+stock_csv+"_winsize"+str(event_size)+'.xlsx'
    writer = pd.ExcelWriter(path, engine='xlsxwriter')

    # Write each dataframe to a different worksheet.
    stocks_data.to_excel(writer, sheet_name='data')
    stocks_metadata.to_excel(writer, sheet_name='meta-data')
    variables.to_excel(writer, sheet_name='AAR')
    oddvalues.to_excel(writer, sheet_name='CAAR_n_Cox')

    # Close the Pandas Excel writer and output the Excel file.
    writer.save()
    print("Eventsize "+str(event_size)+ " completed!")
    

  4%|▍         | 15/381 [00:02<01:10,  5.16it/s]

Stock 9 in row: 27 disregarded. nan values present
Stock 10 in row: 30 disregarded. nan values present
Stock 11 in row: 33 disregarded. nan values present
Stock 12 in row: 36 disregarded. nan values present
Stock 13 in row: 39 disregarded. nan values present


 34%|███▍      | 130/381 [00:33<01:05,  3.83it/s]

Stock 130 in row: 390 disregarded. nan values present


 41%|████▏     | 158/381 [00:39<00:50,  4.41it/s]

...

Stock 346 in row: 1038 disregarded. nan values present
Stock 347 in row: 1041 disregarded. nan values present
Stock 348 in row: 1044 disregarded. nan values present


 99%|█████████▊| 376/381 [02:12<00:03,  1.53it/s]

Stock 376 in row: 1128 disregarded. nan values present


100%|██████████| 381/381 [02:14<00:00,  2.83it/s]


Event 21 completed!
